In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()

In [2]:
type(sc)
pyspark.context.SparkContext

pyspark.context.SparkContext

In [3]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils

In [5]:
raw_data = sc.textFile("daily_show.tsv")

In [6]:
raw_data.take(5)

['YEAR,GoogleKnowlege_Occupation,Show,Group,Raw_Guest_List',
 '1999,actor,1/11/99,Acting,Michael J. Fox',
 '1999,Comedian,1/12/99,Comedy,Sandra Bernhard',
 '1999,television actress,1/13/99,Acting,Tracey Ullman',
 '1999,film actress,1/14/99,Acting,Gillian Anderson']

In [21]:
data_file = "./kddcup.data_10_percent_corrected"

In [22]:
raw_data = sc.textFile(data_file)

In [25]:
raw_data.count()

494021

In [26]:
raw_data.take(5)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,39,39,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,49,49,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.']

In [27]:
a = range(100)

In [28]:
data = sc.parallelize(a)

In [29]:
data.count()

100

In [30]:
data.take(5)

[0, 1, 2, 3, 4]

In [31]:
normal_raw_data = raw_data.filter(lambda x: 'normal.' in x)

In [33]:
from time import time

In [35]:
t0 = time()
normal_count = normal_raw_data.count()
tt = time() - t0
print("There are {} 'normal' interactions".format(normal_count))
print("count completed in {} seconds".format(round(tt, 3)))

There are 97278 'normal' interactions
count completed in 2.432 seconds


In [37]:
from pprint import pprint
csv_data = raw_data.map(lambda x: x.split(","))
t0 = time()
head_rows = csv_data.take(5)
tt = time() - t0
print("Parse completed in {} seconds".format(round(tt, 3)))
pprint(head_rows[0])

Parse completed in 0.166 seconds
['0',
 'tcp',
 'http',
 'SF',
 '181',
 '5450',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '8',
 '8',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '1.00',
 '0.00',
 '0.00',
 '9',
 '9',
 '1.00',
 '0.00',
 '0.11',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 'normal.']


In [38]:
t0 = time()
head_rows = csv_data.take(10000)
tt = time() - t0
print("Parse completed in {} seconds".format(round(tt, 3)))

Parse completed in 1.664 seconds


In [39]:
def parse_interaction(line):
    elems = line.split(",")
    tag = elems[41]
    return(tag, elems)
key_csv_data = raw_data.map(parse_interaction)
head_rows = key_csv_data.take(5)
pprint(head_rows[0])

('normal.',
 ['0',
  'tcp',
  'http',
  'SF',
  '181',
  '5450',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '9',
  '9',
  '1.00',
  '0.00',
  '0.11',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'])


In [40]:
t0 = time()
all_raw_data = raw_data.collect()
tt = time() - t0
print("Data collected in {} seconds".format(round(tt,3)))

Data collected in 13.517 seconds


In [41]:
data_file = './kddcup.data_10_percent_corrected'
raw_data = sc.textFile(data_file)
# parse into key-value pairs
key_csv_data = raw_data.map(parse_interaction)

In [42]:
normal_key_interactions = key_csv_data.filter(lambda x: x[0] == "normal.")

In [43]:
t0 = time()
all_normal = normal_key_interactions.collect()
tt = time() - t0
normal_count = len(all_normal)

In [44]:
print("DAta colected in {} seconds".format(round(tt, 3)))
print("There are {} 'normal' interactions".format(normal_count))

DAta colected in 12.599 seconds
There are 97278 'normal' interactions


In [45]:
data_file = "./kddcup.data_10_percent_corrected"
raw_data = sc.textFile(data_file)

In [46]:
raw_data_sample = raw_data.sample(False, 0.1, 1234)

In [47]:
sample_size = raw_data_sample.count()

In [48]:
total_size = raw_data.count()
print("Sample size is {} of {}".format(sample_size, total_size))

Sample size is 49231 of 494021


In [49]:
from time import time
# transformations to be applied
raw_data_sample_items = raw_data_sample.map(lambda x: x.split(","))
sample_normal_tags = raw_data_sample_items.filter(lambda x: "normal." in x)

In [50]:
t0 = time()
sample_normal_tags_count = sample_normal_tags.count()

In [51]:
tt = time() - t0

In [53]:
sample_normal_ratio = sample_normal_tags_count / float(sample_size)
print("The raio of 'normal' interactions is {}".format(round(sample_normal_ratio, 3)))
print("Count done in {} seconds".format(round(tt, 3)))

The raio of 'normal' interactions is 0.195
Count done in 22.073 seconds


In [54]:
t0 = time()
raw_data_sample = raw_data.takeSample(False, 400000, 1234)
normal_data_sample = [x.split(",") for x in raw_data_sample if "normal." in x]
tt = time() - t0

normal_sample_size = len(normal_data_sample)
normal_ratio = normal_sample_size / 400000.0
print("The ratio of 'normal' interaction is {}".format(normal_ratio))
print("Count done in {} seconds".format(round(tt, 3)))

The ratio of 'normal' interaction is 0.19696
Count done in 14.822 seconds


In [55]:
data_file = './kddcup.data_10_percent_corrected'
raw_data = sc.textFile(data_file)

In [56]:
normal_raw_data = raw_data.filter(lambda x: "normal." in x)

In [57]:
attack_raw_data = raw_data.subtract(normal_raw_data)

In [58]:
from time import time
# count all
t0 = time()
raw_data_count = raw_data.count()
tt = time() - t0
print("All count in {} secs".format(round(tt,3)))

All count in 3.147 secs


In [62]:
# count attacks
t0 = time()
attack_raw_data_count = attack_raw_data.count()
tt = time() - t0
print("Attack count in {} secs".format(round(tt, 3)))

Attack count in 3.502 secs


In [63]:
csv_data = raw_data.map(lambda x: x.split(","))
protocols = csv_data.map(lambda x: x[1]).distinct()
protocols.collect()

['tcp', 'icmp', 'udp']

In [64]:
services = csv_data.map(lambda x: x[2]).distinct()
services.collect()

['login',
 'supdup',
 'netbios_dgm',
 'sunrpc',
 'urp_i',
 'tftp_u',
 'Z39_50',
 'csnet_ns',
 'uucp_path',
 'auth',
 'whois',
 'domain_u',
 'http',
 'urh_i',
 'imap4',
 'pop_3',
 'red_i',
 'pm_dump',
 'rje',
 'netstat',
 'finger',
 'link',
 'name',
 'iso_tsap',
 'hostnames',
 'vmnet',
 'shell',
 'ecr_i',
 'ssh',
 'smtp',
 'ctf',
 'mtp',
 'courier',
 'telnet',
 'netbios_ns',
 'private',
 'gopher',
 'efs',
 'IRC',
 'ftp_data',
 'eco_i',
 'echo',
 'bgp',
 'ldap',
 'nntp',
 'other',
 'netbios_ssn',
 'systat',
 'remote_job',
 'domain',
 'http_443',
 'time',
 'exec',
 'tim_i',
 'ftp',
 'ntp_u',
 'pop_2',
 'klogin',
 'daytime',
 'sql_net',
 'nnsp',
 'uucp',
 'discard',
 'kshell',
 'X11',
 'printer']

In [65]:
product = protocols.cartesian(services).collect()
print("There are {} combinatiosn of probocol X service".format(len(product)))

There are 198 combinatiosn of probocol X service


In [66]:
data_file = "./kddcup.data_10_percent_corrected"
raw_data = sc.textFile(data_file)

In [67]:
csv_data = raw_data.map(lambda x: x.split(","))

normal_csv_data = csv_data.filter(lambda x: x[41] == "normal.")
attack_csv_data = csv_data.filter(lambda x: x[41 != "normal."])

In [68]:
normal_csv_data.take(3)

[['0',
  'tcp',
  'http',
  'SF',
  '181',
  '5450',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '9',
  '9',
  '1.00',
  '0.00',
  '0.11',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'],
 ['0',
  'tcp',
  'http',
  'SF',
  '239',
  '486',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '19',
  '19',
  '1.00',
  '0.00',
  '0.05',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'],
 ['0',
  'tcp',
  'http',
  'SF',
  '235',
  '1337',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '29',
  '29',
  '1.00',
  '0.00',


In [69]:
attack_csv_data.take(1)

[['0',
  'tcp',
  'http',
  'SF',
  '181',
  '5450',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '9',
  '9',
  '1.00',
  '0.00',
  '0.11',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.']]

In [70]:
data_file = "./kddcup.data_10_percent_corrected"
raw_data = sc.textFile(data_file)

In [71]:
csv_data = raw_data.map(lambda x: x.split(","))
key_value_data = csv_data.map(lambda x: (x[41], x))

In [72]:
key_value_data.take(1)

[('normal.',
  ['0',
   'tcp',
   'http',
   'SF',
   '181',
   '5450',
   '0',
   '0',
   '0',
   '0',
   '0',
   '1',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '8',
   '8',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '1.00',
   '0.00',
   '0.00',
   '9',
   '9',
   '1.00',
   '0.00',
   '0.11',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   'normal.'])]

In [73]:
key_value_duration = csv_data.map(lambda x: (x[41], float(x[0])))

In [74]:
durations_by_key = key_value_duration.reduceByKey(lambda x, y: x + y)

In [75]:
durations_by_key.collect()

[('multihop.', 1288.0),
 ('rootkit.', 1008.0),
 ('warezclient.', 627563.0),
 ('ipsweep.', 43.0),
 ('teardrop.', 0.0),
 ('warezmaster.', 301.0),
 ('pod.', 0.0),
 ('perl.', 124.0),
 ('back.', 284.0),
 ('land.', 0.0),
 ('portsweep.', 1991911.0),
 ('guess_passwd.', 144.0),
 ('smurf.', 0.0),
 ('satan.', 64.0),
 ('imap.', 72.0),
 ('ftp_write.', 259.0),
 ('loadmodule.', 326.0),
 ('nmap.', 0.0),
 ('neptune.', 0.0),
 ('buffer_overflow.', 2751.0),
 ('normal.', 21075991.0),
 ('spy.', 636.0),
 ('phf.', 18.0)]

In [76]:
counts_by_key = key_value_data.countByKey()

In [77]:
counts_by_key

defaultdict(int,
            {'back.': 2203,
             'buffer_overflow.': 30,
             'ftp_write.': 8,
             'guess_passwd.': 53,
             'imap.': 12,
             'ipsweep.': 1247,
             'land.': 21,
             'loadmodule.': 9,
             'multihop.': 7,
             'neptune.': 107201,
             'nmap.': 231,
             'normal.': 97278,
             'perl.': 3,
             'phf.': 4,
             'pod.': 264,
             'portsweep.': 1040,
             'rootkit.': 10,
             'satan.': 1589,
             'smurf.': 280790,
             'spy.': 2,
             'teardrop.': 979,
             'warezclient.': 1020,
             'warezmaster.': 20})

In [79]:
sum_counts = key_value_duration.combineByKey(
    (lambda x: (x, 1)),
    (lambda acc, value: (acc[0]+value, acc[1]+1)),
    (lambda acc1, acc2: (acc1[0]+ acc2[0], acc1[1]+ acc2[1]))
    )
sum_counts.collectAsMap()

{'back.': (284.0, 2203),
 'buffer_overflow.': (2751.0, 30),
 'ftp_write.': (259.0, 8),
 'guess_passwd.': (144.0, 53),
 'imap.': (72.0, 12),
 'ipsweep.': (43.0, 1247),
 'land.': (0.0, 21),
 'loadmodule.': (326.0, 9),
 'multihop.': (1288.0, 7),
 'neptune.': (0.0, 107201),
 'nmap.': (0.0, 231),
 'normal.': (21075991.0, 97278),
 'perl.': (124.0, 3),
 'phf.': (18.0, 4),
 'pod.': (0.0, 264),
 'portsweep.': (1991911.0, 1040),
 'rootkit.': (1008.0, 10),
 'satan.': (64.0, 1589),
 'smurf.': (0.0, 280790),
 'spy.': (636.0, 2),
 'teardrop.': (0.0, 979),
 'warezclient.': (627563.0, 1020),
 'warezmaster.': (301.0, 20)}

In [87]:
import numpy as np

def parse_interaction(line):
    line_split = line.split(",")
    # keep just numeric and logical values
    symbolic_indexes = [1,2,3,41]
    clean_line_split = [item for i,item in enumerate(line_split) if i not in symbolic_indexes]
    return np.array([float(x) for x in clean_line_split])

vector_data = raw_data.map(parse_interaction)

In [88]:
from pyspark.mllib.stat import Statistics
from math import sqrt

In [89]:
# Compute column summary statistics
summary = Statistics.colStats(vector_data)

In [91]:
print("Duration Statistics:")
print(" Mean: {}".format(round(summary.mean()[0],3)))


Duration Statistics:
 Mean: 47.979


In [93]:
data_file = './kddcup.data.corrected'
raw_data = sc.textFile(data_file)

print("Train data size is {}".format(raw_data.count()))

Train data size is 4898431


In [94]:
test_data_file = "./corrected"
test_raw_data = sc.textFile(test_data_file)
print("Test data size is {}".format(test_raw_data.count()))

Test data size is 311029


In [95]:
from pyspark.mllib.regression import LabeledPoint
from numpy import array

In [96]:
def parse_interaction(line):
    line_split = line.split(",")
    clean_line_split = line_split[0:1]+line_split[4:41]
    attack = 1.0
    if line_split[41] == 'normal.':
        attack = 0.0
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

In [97]:
training_data = raw_data.map(parse_interaction)

In [98]:
test_data = test_raw_data.map(parse_interaction)

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from time import time

t0 = time()
logit_model = LogisticRegressionWithLBFGS.train(training_data)
tt = time() - t0
round(tt, 3)

https://github.com/jadianes/spark-py-notebooks/blob/master/nb8-mllib-logit/nb8-mllib-logit.ipynb